<a href="https://colab.research.google.com/github/RKDash7/R-K-Dash/blob/main/Blockchain_IoT_Edge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cryptography

In [22]:
import hashlib
import time
import json
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

# Blockchain class
class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        # Create the first block (genesis block)
        self.create_block(previous_hash='1', proof=100)

    def create_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transactions = []  # Reset current transactions
        self.chain.append(block)
        return block

    def add_transaction(self, sender, recipient, amount, signature):
        """ Adds a transaction to the current list of transactions. """
        if not self.verify_signature(sender, signature):
            raise Exception("Invalid signature")

        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'signature': signature
        })
        return self.last_block['index'] + 1

    def proof_of_work(self, last_proof):
        """ Proof of Work algorithm to find a valid nonce. """
        proof = 0
        while not self.valid_proof(last_proof, proof):
            proof += 1
        return proof

    def valid_proof(self, last_proof, proof):
        """ Validate the Proof of Work by checking if the hash starts with '0000'. """
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

    def hash(self, block):
        """ Hash a block with SHA-256. """
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def verify_signature(self, sender, signature):
        """ Verify the signature of a transaction. """
        public_key = sender["public_key"]
        message = f"{sender['id']} sent to {signature['recipient']} {signature['amount']} packets"
        try:
            public_key.verify(
                signature['signature'],
                message.encode(),
                padding.PKCS1v15(),
                hashes.SHA256()
            )
            return True
        except:
            return False

    @property
    def last_block(self):
        return self.chain[-1]

# Digital Signature and RSA Key Generation
def generate_keys():
    """ Generate RSA public and private keys for signing and verification. """
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )

    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_key, public_key, private_pem, public_pem

def sign_transaction(private_key, message):
    """ Sign a message (transaction) using the sender's private key. """
    signature = private_key.sign(
        message.encode(),
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    return signature

# Example usage of the blockchain and transaction
if __name__ == '__main__':
    # Device 1: Generate Keys and Blockchain
    device1_private, device1_public, device1_private_pem, device1_public_pem = generate_keys()
    device1 = {"id": "Node 1", "public_key": device1_public}

    # Device 2: Generate Keys
    device2_private, device2_public, device2_private_pem, device2_public_pem = generate_keys()
    device2 = {"id": "Node 2", "public_key": device2_public}

    # Create a Blockchain instance
    blockchain = Blockchain()

    # Device 1 creates a transaction to send to Device 2
    message = f"{device1['id']} sent to {device2['id']} 100 packets"
    signature = sign_transaction(device1_private, message)

    # Add the transaction to the blockchain (Device 1 sends funds to Device 2)
    blockchain.add_transaction(device1, device2, 100, {'signature': signature, 'recipient': device2['id'], 'amount': 100})

    # Device 1 mines a new block (Proof of Work)
    last_proof = blockchain.last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # Device 1 creates a new block after mining
    previous_hash = blockchain.hash(blockchain.last_block)
    blockchain.create_block(proof, previous_hash)

    print("Blockchain after mining a new block:")
    for block in blockchain.chain:
        print(block)


Blockchain after mining a new block:
{'index': 1, 'timestamp': 1736852933.658166, 'transactions': [], 'proof': 100, 'previous_hash': '1'}
{'index': 2, 'timestamp': 1736852933.7162006, 'transactions': [{'sender': {'id': 'Node 1', 'public_key': <cryptography.hazmat.bindings._rust.openssl.rsa.RSAPublicKey object at 0x7c9f11323470>}, 'recipient': {'id': 'Node 2', 'public_key': <cryptography.hazmat.bindings._rust.openssl.rsa.RSAPublicKey object at 0x7c9f11323430>}, 'amount': 100, 'signature': {'signature': b'cs\xef{_\x98jR|\xe0! s]"\x80\xcc\x8b\xe8\xb0\xc9\x98\xd7\x90(\xf5\x8d-\xcc*\x8c\xa1\x88am\xe1\x81\xbe3\x96\xaf\x93D\xe0=m\xa7\x97\x19e\xda\xe5\xaelD\x7f-\x9c\x1f\xa7\x16}\x97\xbbPfU!\xb9\xb7\n18\x10\xbc!\xef\xe6\xad\xda\x14,\x9ag{\x1c\x1e\x9d"\xd4\xa0\xbc)\x11Y\xfb\xd4\x04\xec\x9fo\x8f\x9e\xc1\x8d\xfe\xc3\'l`_\xd6\x7fL\xdc\xbeZ$(V\xcaD\xe4\x97\x0e\xc1\x85\x98\xf1\xf1[\x12\xa1\x08T%\xc8\x8a\xea\x1dzZ\xc4\x0e*\xaap@\xb7~(\xaf\x94\xe4\x83:\x11]\x9a_%P;\x17\x1c\xb40\x12\xf8_\xc0\xd3\x1f"\t\